In [101]:
import pandas as pd
import plotly.express as px

In [102]:
df = pd.read_csv('data/atf_data.csv')

/Users/arijitsen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (6,11,15,20) have mixed types.Specify dtype option on import or set low_memory=False.



In [103]:
# Drop duplicate records.
print(len(df))
df.drop_duplicates(subset =['license_name', 'lic_type', 'date','premise_street', 'premise_city', 'premise_state', 'premise_zip_code'], keep='first', inplace = True)
print(len(df))

6517466
4904749


In [104]:
# Find remaining duplicate license names for type 7 records in Texas. 

count = 0
for i in set(df['date']):
    dfx = df.loc[((df['date'] == i) & (df['premise_state'] == 'TX') & (df['lic_type'] == 7))].value_counts('license_name').to_frame().rename(columns = {0:'count'}).sort_values('count', ascending=False)
    if len(dfx) > 0 and dfx['count'].iloc[-1] > 1:
        count += 1
    else:
        pass
print(count)
print(len(set(df['date'])))

# 36 out of 42 months have every single license name appearing more than once.

36
42


In [105]:
# Filter to only gun manufacturers, group by state and date and get number of manufacturers in that state in that month.

df2 = df.loc[df['lic_type'] == 7].groupby(['premise_state', 'date'], as_index=False).count()
df2['count'] = df2.max(axis=1)
df2 = df2[['premise_state', 'date', 'count']]
df2

,premise_state,date,count
0,AK,2018-01-01,73
1,AK,2018-02-01,148
2,AK,2018-03-01,150
3,AK,2018-04-01,152
4,AK,2018-05-01,77
...,...,...,...
2180,WY,2022-01-01,316
2181,WY,2022-02-01,159
2182,WY,2022-03-01,324
2183,WY,2022-04-01,163


In [106]:
# Set Texas line to red and all others to grey.
color_discrete_map = {}
for i in set(df['premise_state']):
    if i == 'TX':
        color_discrete_map[i] = 'red'
    else:
        color_discrete_map[i] = 'grey'

# Plot a line chart with data.         
fig = px.line(df2, x='date', y='count',
              range_x=['2018-01-01','2022-05-02'],
              height=800, width=1200,
              color='premise_state', color_discrete_map=color_discrete_map, 
              markers=True, 
              template='simple_white',
              title='<b>Licensed Gun Manufacturers by State and Month</b>')

fig.update_traces(opacity=0.6, line=dict(width=4))
fig.update_yaxes(range=[0, 7000])
fig.update_layout(font_family='Arial')
fig.layout.showlegend = False
fig.show()

In [107]:
# Export HTML of the chart.
fig.write_html("data/gun_manufacturers.html")